In [1]:
import keras

Using TensorFlow backend.


In [2]:
from keras.datasets import mnist

Loaded MNIST dataset consisting of 70000 images of handwritten digits (0 through 9), split into 60000 training samples and 10000 test samples.  The input is 28 x 28 greyscale image with saturation values ranging between 0 and 255, and the output is the digit represented (0 through 9).  

In [3]:
(train_inputs, train_outputs), (test_inputs, test_outputs) = mnist.load_data()

In [4]:
train_inputs.shape

(60000, 28, 28)

In [5]:
test_inputs.shape

(10000, 28, 28)

In [6]:
train_outputs[0: 5]

array([5, 0, 4, 1, 9], dtype=uint8)

In [7]:
train_inputs[0]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
         18,  18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,
          0,   0],
       [  

In [8]:
train_inputs = train_inputs / 255
test_inputs = test_inputs / 255

In [9]:
train_inputs = train_inputs.reshape((-1, 28, 28, 1))
test_inputs = test_inputs.reshape((-1, 28, 28, 1))

In [10]:
train_inputs.shape

(60000, 28, 28, 1)

Scale inputs so that pixel saturation values lie between 0 and 1, then convert each sample to the standard (width, height, depth) format used by CNNs in keras.

In [11]:
from keras.utils import to_categorical

In [12]:
train_outputs = to_categorical(train_outputs)
test_outputs = to_categorical(test_outputs)

In [13]:
train_outputs[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

Outputs are transformed using the to_categorical utility function, mapping a digit k to a vector of length 10 with the k-th entry set to one and the remaining entries set to zero.

In [14]:
validation_inputs = train_inputs[0 : 15000]
validation_outputs = train_outputs[0 : 15000]

In [15]:
reduced_train_inputs = train_inputs[15000 :]
reduced_train_outputs = train_outputs[15000 :]

Set aside 15000 of the 60000 training samples as a validation set.  It would be better to draw this collection randomly from our training data, rather than using the sequential approach above.

In [16]:
from keras import models
from keras import layers

In [17]:
model = models.Sequential()
model.add(layers.Conv2D(32, 3, activation = 'relu', input_shape = (28, 28, 1, )))
model.add(layers.MaxPooling2D(2))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D(2))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(10, activation = 'softmax'))

In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)               

In [18]:
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

Observe the softmax activation function in our final network layer, an appropriate choice for a multiclass classification model. In addition, note the use of the categorical cross-entropy loss function and the accuracy performance metric in the .compile() parameter list.  Other performance metric choices include precision and recall.

In [19]:
EPOCHS = 20

outcome = model.fit(reduced_train_inputs, reduced_train_outputs, 
          batch_size = 512, epochs = EPOCHS, 
          validation_data = (validation_inputs, validation_outputs))

Train on 45000 samples, validate on 15000 samples
Epoch 1/20
16896/45000 [==========>...................] - ETA: 21s - loss: 1.1204 - accuracy: 0.6450

KeyboardInterrupt: 

In [ ]:
outcome.history.keys()

In [ ]:
epochs = range(1, EPOCHS + 1)
accuracy = outcome.history.get('accuracy')
val_accuracy = outcome.history.get('val_accuracy')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(epochs, accuracy, 'bo', label = 'Training Accuracy')
plt.plot(epochs, val_accuracy, 'b', label = 'Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

The plot above shows an increase in both training and validation accuracy over successive epochs.   This behavior can also be seen in the callbacks generated by the .fit() method above.  Ideally, we would train the model for more epochs, until validation accuracy peaked and began to decline (i.e., up to the onset of model overfitting).  We could then combat this overfitting by partially training the model, halting training at an early epoch (say, the twentieth).  We might also reduce overfitting by adding regularization terms to the loss function (see e.g. denseNN_multiclass.ipynb), adding drop-out terms to the network (see e.g. denseNN_binaryclass.ipynb), or decreasing the network size.  

In [ ]:
model.evaluate(test_inputs, test_outputs)